# Generating Cosmic Microwave Background (CMB) temperature anisotropy maps from arbitrary angular power spectrum

In [ ]:
import os
import sys
import numpy as np
import healpy as hp

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
import gencmb
import reconcmb

In [ ]:
# Inuput and output folders
data = '../data/'
out = '../out/'

In [ ]:
# Set axtick dimensions
major_size = 5
minor_size = 3
major_width = minor_width = 1
for tick in ['xtick', 'ytick']:
    mpl.rcParams[f'{tick}.major.size'] = major_size
    mpl.rcParams[f'{tick}.major.width'] = major_width
    mpl.rcParams[f'{tick}.minor.size'] = minor_size
    mpl.rcParams[f'{tick}.minor.width'] = minor_width
mpl.rcParams['text.usetex'] = True

## Calculate CAMB spectrum

In [ ]:
import camb
from camb import model, initialpower

In [ ]:
# Set up a new set of parameters for CAMB
pars = camb.CAMBparams()

In [ ]:
# Use Planck2018 TT,TE,EE+lowE+lensing
pars.set_cosmology(H0=67.36, ombh2=0.02237, omch2=0.1200, mnu=0.06, omk=0, tau=0.0544)
pars.InitPower.set_params(As=2e-9, ns=0.9649, r=0)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [ ]:
%%time
results = camb.get_results(pars)

In [ ]:
%%time
powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')

In [ ]:
DlTT = powers['total'][2:-50][:, 0]
ell = np.arange(2, DlTT.shape[0]+2)
ClTT = DlTT * 2*np.pi / (ell * (ell + 1))

## ... or load input angular power spectrum

In [ ]:
#ell, ClTT, DlTT = gencmb.load_spectrum(fname='../data/CAMB_real_scalCls.dat', lmax=2500)

## Set parameters for SYNFAST

In [ ]:
# Set FWHM and sigma coefficients to match with Planck's
# instruemntal parameters. Since all frequency range have different
# FWHM and sigma values, I only gave here an approximate average for them.
#
# This info can be found here:
# - https://wiki.cosmos.esa.int/planck-legacy-archive/index.php/Beams_LFI
# - https://wiki.cosmos.esa.int/planckpla/index.php/Effective_Beams
fwhm_arcmin = 3
fwhm = np.deg2rad(fwhm_arcmin / 60)

sigma_arcmin = 0.50
sigma = np.deg2rad(sigma_arcmin / 60)

### Generate a random HEALPix dataset using SYNFAST

In [ ]:
maps = gencmb.gen_maps(ClTT, N_SIDE=2048, lmax=None, pol=False, pixwin=False, fwhm=fwhm, sigma=sigma)

### Map HEALPix dataset to a geographical projection and visualize it

In [ ]:
hpx_proj = reconcmb.get_projection(maps, proj='moll', N_SIDE=2048)

In [ ]:
reconcmb.plot_cmb(proj=hpx_proj, cmap=None, c_min=-300, c_max=300,
                  save=True, save_filename='CMB_HEALPix_sim_5.png')

### Reconstruct angular power spectrum from generated dataset

In [ ]:
ell, Cl, Dl = reconcmb.cmb_spectrum(maps, lmax=2500, alm=False)

In [ ]:
reconcmb.plot_spectrum(ell, Dl, DlTT,
                       save=False, save_filename='default_name_spectrum')